In [284]:

from sklearn.ensemble import RandomForestClassifier
from agoda_cancellation_estimator import AgodaCancellationEstimator
from agoda_cancellation_prediction import load_data
import pandas as pd
import numpy as np

In [285]:
full_data = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\agoda_cancellation_train.csv").dropna().drop_duplicates()
test_data_w1 = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\test_set_week_1.csv").dropna().drop_duplicates()
full_data.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
16,-9216487696802692260,2018-08-28 22:40:00,2018-09-29 00:00:00,2018-09-30 00:00:00,2559209,JP,2017-07-21 18:32:00,3.0,Hotel,Pay Now,...,0.0,0.0,1.0,0.0,0.0,2018-08-28,5092,366.0,618.0,1403
19,-9215494322833174318,2018-05-27 17:15:00,2018-08-20 00:00:00,2018-08-21 00:00:00,11934,NZ,2009-06-28 02:02:00,4.0,Hotel,Pay Later,...,0.0,0.0,1.0,0.0,0.0,2018-07-13,3450,159.0,232.0,2158
22,-9215072291520077707,2018-05-31 22:43:00,2018-09-21 00:00:00,2018-09-22 00:00:00,108170,TH,2009-06-28 02:02:00,3.0,Hotel,Pay Later,...,1.0,1.0,0.0,0.0,0.0,2018-06-19,662,442.0,600.0,447
98,-9191194571019688471,2018-06-18 09:47:00,2018-09-07 00:00:00,2018-09-09 00:00:00,2411232,US,2017-08-30 00:49:00,4.0,Hotel,Pay Later,...,0.0,0.0,0.0,0.0,0.0,2018-09-05,5540,753.0,548.0,202
137,-9179243281126879585,2018-03-13 16:09:00,2018-08-24 00:00:00,2018-08-28 00:00:00,1899389,JP,2017-01-27 09:03:00,4.0,Hotel,Pay Later,...,1.0,0.0,1.0,0.0,0.0,2018-03-23,643,95.0,254.0,2249


In [286]:

full_data.accommadation_type_name = full_data.accommadation_type_name.astype('category')
full_data.customer_nationality = full_data.customer_nationality.astype(str).astype('category')
full_data.hotel_id = full_data.hotel_id.astype('category')
full_data.charge_option = full_data.charge_option.astype('category')
full_data.cancellation_policy_code = full_data.cancellation_policy_code.astype('category')
full_data['customer_nationality_codes'] = full_data.customer_nationality.cat.codes
full_data['accommadation_type_name_codes'] = full_data.accommadation_type_name.cat.codes
full_data['hotel_id_codes'] = full_data.hotel_id.cat.codes
full_data['charge_option_codes'] = full_data.charge_option.cat.codes
full_data['cancellation_policy_code_codes'] = full_data.cancellation_policy_code.cat.codes
full_data['cancellation_policy_code_codes']

16       145
19        66
22        44
98        68
137       56
        ... 
58548     41
58556     47
58572     40
58602     66
58626     45
Name: cancellation_policy_code_codes, Length: 2365, dtype: int16

In [287]:
full_data['label'] = full_data.cancellation_datetime.map(lambda x: x > '2018-08-13' and x < '2018-08-20')
full_data.loc[:, ['cancellation_datetime', 'label']]

,cancellation_datetime,label
16,2018-08-28,False
19,2018-07-13,False
22,2018-06-19,False
98,2018-09-05,False
137,2018-03-23,False
...,...,...
58548,2018-03-09,False
58556,2018-07-22,False
58572,2018-07-18,False
58602,2018-05-20,False


In [288]:
# split the data into training set and test set
train_ratio = 0.75
rand_nums = np.random.random_sample(len(full_data))
full_data['train'] =  rand_nums <= train_ratio
full_data.train.value_counts()
train_df = full_data[full_data.train]
test_df = full_data[~full_data.train]
len(train_df), len(test_df), len(full_data)

(1755, 610, 2365)

In [289]:
features_cols = ["hotel_id",
                "accommadation_type_name_codes", "charge_option_codes", "no_of_room",
                "hotel_star_rating",
                "customer_nationality_codes",
                 'cancellation_policy_code_codes']
train_features = train_df[features_cols]
test_features = test_df[features_cols]
train_features.head()


,hotel_id,accommadation_type_name_codes,charge_option_codes,no_of_room,hotel_star_rating,customer_nationality_codes,cancellation_policy_code_codes
16,2559209,5,1,1,3.0,54,145
19,11934,5,0,1,4.0,36,66
22,108170,5,0,1,3.0,55,44
98,2411232,5,0,1,4.0,61,68
137,1899389,5,0,1,4.0,20,56


In [290]:
# features = full_data[[#"h_booking_id",
#                       "hotel_id",
#                       "accommadation_type_name_codes",
#                       "hotel_star_rating",
#                       "customer_nationality_codes"]]
# features.head()


In [291]:
train_labels = train_df["cancellation_datetime"]
test_labels = test_df["cancellation_datetime"]
train_labels.head()

16     2018-08-28
19     2018-07-13
22     2018-06-19
98     2018-09-05
137    2018-03-23
Name: cancellation_datetime, dtype: object

In [292]:
full_data['cancellation_datetime']

16       2018-08-28
19       2018-07-13
22       2018-06-19
98       2018-09-05
137      2018-03-23
            ...    
58548    2018-03-09
58556    2018-07-22
58572    2018-07-18
58602    2018-05-20
58626    2018-08-23
Name: cancellation_datetime, Length: 2365, dtype: object

In [293]:
random_classifier = RandomForestClassifier()

In [294]:
y_train = train_df.label.values
X_train = train_features.values
y_test = test_df.label.values
X_test = test_features.values
X_train

array([[2.559209e+06, 5.000000e+00, 1.000000e+00, ..., 3.000000e+00,
        5.400000e+01, 1.450000e+02],
       [1.193400e+04, 5.000000e+00, 0.000000e+00, ..., 4.000000e+00,
        3.600000e+01, 6.600000e+01],
       [1.081700e+05, 5.000000e+00, 0.000000e+00, ..., 3.000000e+00,
        5.500000e+01, 4.400000e+01],
       ...,
       [3.091081e+06, 5.000000e+00, 1.000000e+00, ..., 5.000000e+00,
        2.100000e+01, 4.000000e+01],
       [4.000867e+06, 5.000000e+00, 1.000000e+00, ..., 3.000000e+00,
        1.000000e+00, 6.600000e+01],
       [4.515765e+06, 5.000000e+00, 0.000000e+00, ..., 2.500000e+00,
        2.600000e+01, 4.500000e+01]])

In [295]:
model = random_classifier.fit(X_train,y_train)

In [296]:
# how many orders the model predicted were cancelled, on a data set he was trained on
np.bincount(model.predict(X_train))

array([1672,   83], dtype=int64)

In [297]:
np.bincount(y_train)

array([1665,   90], dtype=int64)

In [298]:
model.predict(X_test)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False,  True,

In [299]:
# how many orders the model predicted were cancelled, on a data set he was not trained on
np.bincount(model.predict(X_test))

array([595,  15], dtype=int64)

In [300]:
# how many orders got actually cancelled
np.bincount(y_test)

array([577,  33], dtype=int64)

In [301]:
np.bincount(model.predict(X_test) & y_test)

array([604,   6], dtype=int64)

In [302]:
np.bincount(model.predict(X_train) & y_train)

array([1673,   82], dtype=int64)

In [303]:
full_data.label.value_counts()

False    2242
True      123
Name: label, dtype: int64

In [304]:
full_data[(full_data.cancellation_datetime > '2018-08-07') & (full_data.cancellation_datetime < '2018-08-20')]

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,hotel_brand_code,hotel_chain_code,hotel_city_code,customer_nationality_codes,accommadation_type_name_codes,hotel_id_codes,charge_option_codes,cancellation_policy_code_codes,label,train
138,-9178271518124899741,2018-06-13 01:39:00,2018-09-20 00:00:00,2018-09-24 00:00:00,2863,GB,2011-08-31 16:49:00,3.0,Hostel,Pay Now,...,131.0,271.0,2538,58,4,23,1,68,False,True
200,-9158997573325902499,2018-08-19 06:24:00,2018-09-15 00:00:00,2018-09-17 00:00:00,3129374,AU,2017-11-06 16:37:00,4.5,Hotel,Pay Now,...,63.0,291.0,1584,1,5,1347,1,40,True,False
225,-9150457930144649121,2018-07-24 01:47:00,2018-08-18 00:00:00,2018-08-19 00:00:00,1602888,IN,2017-03-17 11:34:00,5.0,Hotel,Pay Later,...,282.0,55.0,2796,21,5,876,0,66,False,False
235,-9148315339177572723,2018-08-13 22:49:00,2018-08-18 00:00:00,2018-08-19 00:00:00,4910396,MY,2018-04-13 19:52:00,2.0,Hotel,Pay Later,...,745.0,94.0,2797,32,5,1597,0,45,True,False
412,-9093276767136234643,2018-06-27 23:39:00,2018-09-19 00:00:00,2018-09-20 00:00:00,168985,JP,2009-12-19 08:20:00,3.5,Hotel,Pay Later,...,918.0,587.0,1014,58,5,349,0,49,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56263,8466705461484802320,2018-07-27 16:15:00,2018-08-24 00:00:00,2018-08-26 00:00:00,1105052,JP,2015-11-25 11:59:00,3.0,Hotel,Pay Later,...,922.0,587.0,183,58,5,770,0,114,True,True
56332,8488847189900316785,2018-07-10 12:45:00,2018-08-25 00:00:00,2018-08-26 00:00:00,9358,JP,2009-06-28 02:02:00,4.0,Hotel,Pay Later,...,463.0,103.0,1403,32,5,69,0,45,False,True
56435,8520771494965438804,2018-08-18 21:31:00,2018-08-20 00:00:00,2018-08-21 00:00:00,240484,JP,2010-11-18 07:08:00,3.5,Hotel,Pay Now,...,777.0,181.0,1783,20,5,427,1,38,True,True
56514,8546579788317624088,2018-08-02 11:58:00,2018-09-13 00:00:00,2018-09-18 00:00:00,3320368,TH,2017-11-10 09:25:00,4.0,Hotel,Pay Later,...,278.0,473.0,2477,54,5,1450,0,132,True,True


In [305]:
test_data_w1 = pd.read_csv(r"C:\Users\97250\PycharmProjects\IML.HUJI\datasets\test_set_week_1.csv").dropna().drop_duplicates()
test_data_w1.head()

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_latecheckin,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
2,-9217919187333201744,2018-11-25 21:44:00,2019-04-22 00:00:00,2019-04-23 00:00:00,400322,JP,2012-10-02 13:54:00,3.0,Hotel,Pay Later,...,0.0,0.0,0.0,1.0,0.0,0.0,2884,193.0,344.0,2264
3,-9215699014988905507,2018-11-20 20:20:00,2018-12-16 00:00:00,2018-12-20 00:00:00,3135771,PH,2017-11-06 17:05:00,5.0,Resort,Pay Now,...,0.0,1.0,1.0,0.0,1.0,0.0,2532,415.0,291.0,1405
5,-9209328318584438243,2018-11-07 18:33:00,2018-12-21 00:00:00,2018-12-22 00:00:00,10602,SG,2014-05-20 08:05:00,5.0,Hotel,Pay Now,...,0.0,0.0,0.0,1.0,0.0,0.0,2868,870.0,551.0,437
10,-9206569785675213125,2018-11-18 12:54:00,2018-12-28 00:00:00,2018-12-29 00:00:00,2559374,HK,2017-07-20 15:36:00,4.0,Hotel,Pay Later,...,0.0,1.0,0.0,0.0,0.0,0.0,1633,613.0,355.0,142
27,-9162547391520652865,2018-11-17 21:38:00,2018-12-31 00:00:00,2019-01-02 00:00:00,185040,JP,2010-05-03 08:59:00,3.0,Hotel,Pay Later,...,0.0,0.0,0.0,0.0,0.0,0.0,4463,658.0,50.0,2567


In [306]:
# convert accomodation type name to codes